In [13]:
import rioxarray
import geopandas
import leafmap
import os
import datetime
#from osgeo import gdal
from tqdm import tqdm
#import cdsapi
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime
import rasterio
import pandas as pd

In [14]:
grib_file = "era_5_evap_soil_moist.grib"
dest_file = "data/era_5_evap_soil_tiff/" + "era_5_evap_soil_moist.tiff"
dest_dir = "data/era_5_evap_soil_tiff"

In [15]:

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'grib',
        'variable': [
            'evaporation', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2',
            'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4',
        ],
        'year': [
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020', '2021', '2022',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'area': [
            51.17, 18.4, 50.47,
            19.68,
        ],
        'product_type': 'monthly_averaged_reanalysis',
        'time': '00:00',
    },
    
    grib_file)

NameError: name 'cdsapi' is not defined

In [ ]:
grib_f = "data/" + grib_file

In [ ]:
src_ds = gdal.Open(grib_f)
bands = [] # Set up array for gdal.Translate(). 
if src_ds is not None:
    bandNum = src_ds.RasterCount # Get band count
for i in range(bandNum+1): # Update array based on band count
    if (i==0): #gdal starts band counts at 1, not 0 like the Python for loop does.
        pass
    else:
        bands.append(i)

# Open output format driver
out_form= "GTiff"

# Output to new format using gdal.Translate. See https://gdal.org/python/ for osgeo.gdal.Translate options.
dst_ds = gdal.Translate(dest_file, src_ds, format=out_form, bandList=bands)

# Properly close the datasets to flush to disk
dst_ds = None
src_ds = None

Warning 1: data/era_5_evap_soil_tiff/era_5_evap_soil_moist.tiff: Metadata exceeding 32000 bytes cannot be written into GeoTIFF. Transferred to PAM instead.


In [ ]:
src_ds = gdal.Open(grib_f)

In [ ]:
ds = gdal.Open(grib_f)

In [ ]:
ds.GetRasterBand(5).GetMetadata()

{'GRIB_COMMENT': 'Volumetric soil water layer 4 [m^3/m^3]',
 'GRIB_ELEMENT': 'SWVL4',
 'GRIB_FORECAST_SECONDS': '0 sec',
 'GRIB_REF_TIME': '  1009843200 sec UTC',
 'GRIB_SHORT_NAME': '100-255-DBLY',
 'GRIB_UNIT': '[m^3/m^3]',
 'GRIB_VALID_TIME': '  1009843200 sec UTC'}

In [ ]:
rootGroup = src_ds.GetRootGroup()
rootGroup
var = rootGroup.OpenMDArray("Evaporation")

NameError: name 'src_ds' is not defined

In [ ]:
rasterBand = ds.GetRasterBand(2)
meta = rasterBand.GetMetadata()
dt = datetime.datetime.fromtimestamp(int(meta['GRIB_VALID_TIME'].lstrip()[:-8]))
dt
meta

{'GRIB_COMMENT': 'Volumetric soil water layer 1 [m^3/m^3]',
 'GRIB_ELEMENT': 'SWVL1',
 'GRIB_FORECAST_SECONDS': '0 sec',
 'GRIB_REF_TIME': '  1009843200 sec UTC',
 'GRIB_SHORT_NAME': '0-7-DBLY',
 'GRIB_UNIT': '[m^3/m^3]',
 'GRIB_VALID_TIME': '  1009843200 sec UTC'}

In [ ]:
def get_file_title(element):
    if element.find("E") != -1:
        return "evaporation/" + element + "_"
    if element.find("SWVL1") != -1:
        return "vol_soil_lvl_1/" + element + "_"
    if element.find("SWVL2") != -1:
        return "vol_soil_lvl_2/" + element + "_"
    if element.find("SWVL3") != -1:
        return "vol_soil_lvl_3/" + element + "_"
    if element.find("SWVL4") != -1:
        return "vol_soil_lvl_4/" + element + "_"

In [ ]:
def prepare_datetime(date):
    if date.strftime("%d") != "01":
        date = date + datetime.timedelta(days=1)
    return date.strftime("%Y-%m-%d")

In [ ]:
out_form= "GTiff"
for i in range(1, ds.RasterCount+1):
    rasterBand = ds.GetRasterBand(i)
    meta = rasterBand.GetMetadata()
    dt = prepare_datetime(datetime.datetime.fromtimestamp(int(meta['GRIB_VALID_TIME'].lstrip()[:-8])))
    filename = get_file_title(meta['GRIB_ELEMENT']) + dt + ".tif"
    filename = os.path.join(dest_dir, filename)
    dst_ds = gdal.Translate(filename, ds, format=out_form, bandList=[i])
    dst_ds = None
    

In [ ]:
f = "evaporation/E_2002-01-01.tif"
f = os.path.join(dest_dir, f)

In [ ]:
aoi_dir = "data/aoi.geojson"
geodf = geopandas.read_file(aoi_dir)
data = rasterio.open(f)
data = rioxarray.open_rasterio(data)[0]
data = data.rio.write_crs("EPSG:4326")
data = data.astype("float32")
data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
data = data.rio.write_crs("EPSG:4326")

RasterioIOError: E_2004-08-01.tif: No such file or directory

In [ ]:
data

<xarray.DataArray (y: 3, x: 6)>
array([[-0.00027116, -0.0002358 , -0.00022406, -0.00023284, -0.00024393,
        -0.00025166],
       [-0.00028515, -0.00024109, -0.00021623, -0.00021149, -0.0002333 ,
        -0.00026857],
       [        nan,         nan,         nan, -0.00021771, -0.00021271,
                nan]], dtype=float32)
Coordinates:
    band         int64 1
  * x            (x) float64 18.4 18.65 18.9 19.15 19.4 19.65
  * y            (y) float64 50.97 50.72 50.47
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:          Area
    GRIB_COMMENT:           Evaporation (of water) [m]
    GRIB_ELEMENT:           E
    GRIB_FORECAST_SECONDS:  0 sec
    GRIB_REF_TIME:          1009821600 sec UTC
    GRIB_SHORT_NAME:        0-SFC
    GRIB_UNIT:              [m]
    GRIB_VALID_TIME:        1009821600 sec UTC
    scale_factor:           1.0
    add_offset:             0.0
    long_name:              0[-] SFC (Ground or water surface)

In [17]:
evaporation_dir = os.path.join(dest_dir, "evaporation")
file_list = os.listdir(evaporation_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(evaporation_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    datetime = f[2:-4]
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_evaporation.pickle")

In [ ]:
df.iloc[0].value.shape

(104,)

In [12]:
aoi_dir = "data/aoi.geojson"
geodf = geopandas.read_file(aoi_dir)

In [18]:
vol_soil_lvl_1_dir = os.path.join(dest_dir, "vol_soil_lvl_1")
file_list = os.listdir(vol_soil_lvl_1_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_1_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    datetime = f[6:-4]
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_1.pickle")

In [19]:
df.iloc[0].value.shape

(104,)

In [19]:
vol_soil_lvl_2_dir = os.path.join(dest_dir, "vol_soil_lvl_2")
file_list = os.listdir(vol_soil_lvl_2_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_2_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    datetime = f[6:-4]
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_2.pickle")

In [20]:
vol_soil_lvl_3_dir = os.path.join(dest_dir, "vol_soil_lvl_3")
file_list = os.listdir(vol_soil_lvl_3_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_3_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    datetime = f[6:-4]
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_3.pickle")

In [22]:
vol_soil_lvl_4_dir = os.path.join(dest_dir, "vol_soil_lvl_4")
file_list = os.listdir(vol_soil_lvl_4_dir)

df = pd.DataFrame(columns = ["date", "value"])

for f in file_list: 
    input_raster = os.path.join(vol_soil_lvl_4_dir, f)
    data = rasterio.open(input_raster)
    data = rioxarray.open_rasterio(data)[0]
    data = data.rio.write_crs("EPSG:4326")
    data = data.astype("float32")
    data = data.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    data_upsampled = data.rio.reproject(
        data.rio.crs,
        shape=(8, 13),
    )
    data_upsampled = data_upsampled.rio.write_crs("EPSG:4326")
    masked = data_upsampled.to_masked_array()
    datetime = f[6:-4]
    df = df.append({'date': datetime, 'value': masked.data}, ignore_index=True)
    
df = df.sort_values(by="date", ignore_index=True)
df.to_pickle("data/era5_vol_soil_lvl_4.pickle")

In [25]:
df.iloc[0]["mask"]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True, False, False, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
       False, False, False,  True,  True])